# Introduction - The Coffee Shop Problem (Toronto) <a name="I"></a>

One of the most common establishments that can be found within any neighborhood of the city of Toronto is coffee shops. Although this shows there is a great market for this type of establishment, it also shows that competition would be fierce for a new business.

The problem is as follows, you are approached by an entrepeneur interested in starting their first small coffee shop. They are asking you to leverage available data to consult them on the most reasonable location within the city of Toronto to start their business. Key factors to take into account are population, and the presence of competition. Assuming cost of operation due to the location is not a concern, where should the entrepeneur open their coffee shop?

The intended audience for this report are entrepeneurs potentials hoping to make the same decision as the hypthetical one mentioned above, using the analysis provided to help their decision. Secondary to this, other data scientists and the curious may also find this report useful as an insight into the business demographics of neighborhoods within their city.

* [Introduction](#I)
* [Data](#D)
* [Methodology](#M)
* [Analysis](#A)
* [Results & Discussion](#RD)
* [Conclusion](#C)

# Data - Wikipedia, Stat Canada and Foursquare API <a name="D"></a>

This analysis will be performed with the list of postal codes of Toronto provided by wikipedia here:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

The 2016 Canadian census data on forward sortation areas provided here (population by postal code):
https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/comprehensive.cfm

The foursquare API will also be used in order to makes sense of the postal codes provided by wikipedia by accessing nearby venues within each neighborhood in order to identify competition in the area.

The above data and API will be used to gether to create a map visualizing viable neighborhoods in toronto and their potential value as a location for a new coffee shop.

# Methodology - Working with the data <a name="M"></a>

In this report we will focus on finding prime neighborhood locations within Toronto featuring both high populations, and low frequency of coffee shops, maximizing demand and minimizing current supply.

**First** we will collect necessary data through wikipedia and  Stat Canada to create a master dataframe containing a comprehensive list of all neighborhoods in Toronto, along with their respective coordinates for later use, population, and name/borough.

**Second** we will do some visualization work with our preliminary data using folium in order to visualize the population of each neighborhood on a single map.

**Third and finally** we will start working with the foursquare API to augment our master dataframe to include the top 3 most common types of categories in each neighborhood and whether or not a coffee shops dominance exists.

## 1 - Package Imports and Webscraping

### 1.1 - Package imports

In [1]:
from bs4 import BeautifulSoup
import requests

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries Imported!")

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries Imported!


### 1.2 - Initialize a postal information dataframe by webscraping the wikipedia page for neighborhoods in Toronto.

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table_contents = []
table = soup.find('table')

for cell in table.findAll('td'):
    
    content = {}
    
    if cell.span.text == "Not assigned":
        pass
    else:
        content['Postal'] = cell.p.text[:3]
        content['Borough'] = (cell.span.text).split("(")[0]
        content['Neighborhood'] = (((((cell.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(content)
        
df_geo_info = pd.DataFrame(table_contents)
df_geo_info['Borough'] = df_geo_info['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df_geo_info = df_geo_info.sort_values(by=['Postal'])
df_geo_info = df_geo_info.reset_index()
df_geo_info = df_geo_info.drop(columns=['index'])
df_geo_info = df_geo_info.rename(columns={"Postal":"Postal Code"})
df_geo_info.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 1.3 - Import Geospatial Coordinates and create

In [3]:
df_geo_coor = pd.read_csv("./Geospatial_Coordinates.csv")
df_geo_coor.sort_values(by=['Postal Code'])
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 1.4 - Import Census data and create dataframe

In [4]:
df_geo_pop = pd.read_csv("./Forward_Sortation_Areas.csv")
df_geo_pop = df_geo_pop[df_geo_pop['Geographic code'].str.startswith("M")]
df_geo_pop = df_geo_pop.reset_index()
df_geo_pop = df_geo_pop.drop(columns=['index', 'Geographic name', 'Province or territory', 'Incompletely enumerated Indian reserves and Indian settlements, 2016', 'Total private dwellings, 2016', 'Private dwellings occupied by usual residents, 2016'], axis=1)
df_geo_pop = df_geo_pop.rename(columns={"Geographic code": "Postal Code", "Population, 2016": "Population"})
df_geo_pop.sort_values(by=['Postal Code'])
df_geo_pop.head()

,Postal Code,Population
0,M1B,66108.0
1,M1C,35626.0
2,M1E,46943.0
3,M1G,29690.0
4,M1H,24383.0


### 1.5 - Even out the data and merge dataframe into a master

Note that the geo population dataframe contains just one less row than the other two dataframes, located at index 86, to solve this we simply remove it from the other two. Since no analysis could be done in this neighborhood using the collected data, we ignore it.

In [5]:
df_geo_coor = df_geo_coor.drop([86])
df_geo_coor = df_geo_coor.reset_index()
df_geo_coor = df_geo_coor.drop(columns=['index'])


df_geo_info = df_geo_info.drop([86])
df_geo_info = df_geo_info.reset_index()
df_geo_info = df_geo_info.drop(columns=['index'])

Now double check the shapes, note all have 102 rows (postal codes) so we're good to move on now

In [6]:
df_geo_info.shape

(102, 3)

In [7]:
df_geo_coor.shape

(102, 3)

In [8]:
df_geo_pop.shape

(102, 2)

Merge all three key dataframes into the master. The master dataframe "df" will be the primary data to be used for the rest of the analysis

In [9]:
df = df_geo_info.merge(df_geo_coor, on='Postal Code')
df = df_geo_pop.merge(df, on='Postal Code')
df.head()

,Postal Code,Population,Borough,Neighborhood,Latitude,Longitude
0,M1B,66108.0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,35626.0,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,46943.0,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,29690.0,Scarborough,Woburn,43.770992,-79.216917
4,M1H,24383.0,Scarborough,Cedarbrae,43.773136,-79.239476


## 2 - Preparation With Geopy and Folium

### 2.1 - Initializing

In [10]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto city are', latitude, longitude)

The coordinates of Toronto city are 43.6534817 -79.3839347


### 2.2 - A simple mapping of Toronto Neighborhoods With Folium
Just to test the folium package and demonstrate a visualization of our current dataframe, mapp all corrdiantes stored on a folium map

In [11]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)

for index, row in df.iterrows():
    marker = folium.CircleMarker(
        color = 'blue',
        fill = True,
        fill_color = '3186CC',
        fill_opacity = 0.4,
        location = [row['Latitude'], row['Longitude']],
        popup = folium.Popup("{} {}".format(row['Postal Code'], row['Neighborhood']), parse_html=True),
        radius = 5)
    marker.add_to(map)
    
map

### 2.3 - Mapping neighborhoods to visualize population

Here we see markers identifying each neighborhood in the Toronto area to be analyzed. Let's take it one step further by computing the radius of each marker to be proportional to their population, this will help visualize the more ideal neighborhoods to potentially open a coffee shop based on customer volume.

Now map markers with a larger normalized radius represent areas of higher population and a potentially more lucrative consumer base.

In [12]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)

for index, row in df.iterrows():
    marker = folium.CircleMarker(
        color = 'blue',
        fill = True,
        fill_color = '3186CC',
        fill_opacity = 0.4,
        location = [row['Latitude'], row['Longitude']],
        popup = folium.Popup("{} {}".format(row['Postal Code'], row['Neighborhood']), parse_html=True),
        radius = 10*((row['Population'] - df['Population'].min() ) / ( df['Population'].max() - df['Population'].min())))
    marker.add_to(map)
    
map

## 3 - Foursquare API and most common venues per neighborhood

### 3.1 - Define Foursquare credentials

In [13]:
CLIENT_ID = '4MZYDV4UF5M3MZ2JY5Q5A1Q0JKZCXPIEFM3YQ3I3PEEKBUFO' # your Foursquare ID
CLIENT_SECRET = 'UVSWDW3KZVML0BSETIAQU3FR4JAOGU0QZ5AAQGKN1CBJNFFP' # your Foursquare Secret
VERSION = '20210710' # Foursquare API version
LIMIT = 1000 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4MZYDV4UF5M3MZ2JY5Q5A1Q0JKZCXPIEFM3YQ3I3PEEKBUFO
CLIENT_SECRET:UVSWDW3KZVML0BSETIAQU3FR4JAOGU0QZ5AAQGKN1CBJNFFP


### 3.2 - Define a function to get nearby venues for given coordinates

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Define a new dataframe, associating all found venues with their respective postal code, category, and coordinates.

In [15]:
df_venues = getNearbyVenues(names=df['Postal Code'], latitudes=df['Latitude'], longitudes=df['Longitude'])
df_venues.head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


one hot encode venue categories per postal code, then compute their frequency

In [16]:
# one hot encoding
toronto_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
toronto_onehot['Postal Code'] = df_venues['Postal Code'] 

# move postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

first_col = toronto_onehot.pop('Postal Code')
toronto_onehot.insert(0, 'Postal Code', first_col)

toronto_venue_frequency = toronto_onehot.groupby('Postal Code').mean().reset_index()

toronto_venue_frequency.head()

,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### 3.3 - Finding primary venues per neighborhood (where is the competition?)
Discover the top 3 most common venue categories for each neighborhood in a new dataframe

In [17]:
def return_most_common_venues(row, num_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_venues]

num_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_venue_frequency['Postal Code']

for ind in np.arange(toronto_venue_frequency.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_venue_frequency.iloc[ind, :], num_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1B,Fast Food Restaurant,Accessories Store,Museum
1,M1C,Bar,Accessories Store,Modern European Restaurant
2,M1E,Medical Center,Restaurant,Breakfast Spot
3,M1G,Coffee Shop,Pharmacy,Korean BBQ Restaurant
4,M1H,Bakery,Fried Chicken Joint,Gas Station


Note that the foursquare API missed some postal codes in its response (M1X and M9A). So again, we'll just drop these.

In [18]:
df = df.drop([16])
df = df.drop([92])
df = df.reset_index()
df = df.drop(columns='index')

### 3.4 - Final Dataframe before analysis
merge this new dataframe to the previousmaster frame.
Below is the final version of the dataframe necessary for our analysis. With data on population and the 3 most common venue types, we can make an educated decision on the best neighborhood in Toronto to open a new coffee shop.

In [19]:
df = df.merge(neighborhoods_venues_sorted, on='Postal Code')
df.head()

,Postal Code,Population,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1B,66108.0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Fast Food Restaurant,Accessories Store,Museum
1,M1C,35626.0,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Bar,Accessories Store,Modern European Restaurant
2,M1E,46943.0,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Medical Center,Restaurant,Breakfast Spot
3,M1G,29690.0,Scarborough,Woburn,43.770992,-79.216917,Coffee Shop,Pharmacy,Korean BBQ Restaurant
4,M1H,24383.0,Scarborough,Cedarbrae,43.773136,-79.239476,Bakery,Fried Chicken Joint,Gas Station


### 3.5 - One final visualization with folium
Create one final map to visualize our fully prepped data. Now we will color markers based on their most common venue type, on top of computing their radius based on population. Markers in green indicate coffee shops are not in the top 3 most common venue types in that neighborhood, yellow means they are the 3rd most common, orange means 2nd, and red means coffee shops are the most common venue type there.

In [20]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)

def map_frame(df):
    for index, row in df.iterrows():
        if row['1st Most Common Venue'] == 'Coffee Shop': fill = 'red'
        elif row['2nd Most Common Venue'] == 'Coffee Shop': fill = 'orange'
        elif row['3rd Most Common Venue'] == 'Coffee Shop': fill = 'yellow'
        else: fill = 'lightgreen'

        marker = folium.CircleMarker(
            color = 'black',
            fill = True,
            fill_color = fill,
            fill_opacity = 1.0,
            location = [row['Latitude'], row['Longitude']],
            popup = folium.Popup("{} {}".format(row['Postal Code'], row['Neighborhood']), parse_html=True),
            radius = 10*((row['Population'] - df['Population'].min() ) / ( df['Population'].max() - df['Population'].min())))
        marker.add_to(map)

map_frame(df)
map

This map and the info it presents will be the basis of our conclusion for this problem.

# Analysis - Where do we open shop? <a name="A"></a>

First we'll perform some exploration using our final master frame, along with some visualizations

## Analysis A - Drop neighborhoods with coffee shops as their primary venue type
The first obvious step is to drop any neighborhood already dominated by coffee shops from our options.

Lets double check the shape of our data prior to dropping some candidates

First, our unaltered data

In [21]:
df.shape

(100, 9)

Next, after dropping bad locations

In [22]:
df[~df['1st Most Common Venue'].isin(['Coffee Shop'])].shape

(81, 9)

Update the dataframe

In [23]:
df = df[~df['1st Most Common Venue'].isin(['Coffee Shop'])]
df = df.reset_index()
df = df.drop(columns='index')
df.head()

,Postal Code,Population,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1B,66108.0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Fast Food Restaurant,Accessories Store,Museum
1,M1C,35626.0,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Bar,Accessories Store,Modern European Restaurant
2,M1E,46943.0,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Medical Center,Restaurant,Breakfast Spot
3,M1H,24383.0,Scarborough,Cedarbrae,43.773136,-79.239476,Bakery,Fried Chicken Joint,Gas Station
4,M1J,36699.0,Scarborough,Scarborough Village,43.744734,-79.239476,Playground,Accessories Store,Miscellaneous Shop


In [24]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)
map_frame(df)
map

As you can see we've lost a number of neighborhoods already, lowering our options from 100 potential neighborhoods down to 81, as demonstrated above. However, it's worth noting that we still have a great number of neighborhoods with populations on the larger side left to choose from

## Analysis B - Drop neighborhoods with coffee shops as their secondary venue type
The first obvious step is to drop any neighborhood already dominated by coffee shops from our options.

Lets Check shapes

First, our data after Analysis A

In [25]:
df.shape

(81, 9)

Next, after dropping bad locations

In [27]:
df[~df['2nd Most Common Venue'].isin(['Coffee Shop'])].shape

(73, 9)

Update the dataframe again.

In [28]:
df = df[~df['2nd Most Common Venue'].isin(['Coffee Shop'])]
df = df.reset_index()
df = df.drop(columns='index')
df.head()

,Postal Code,Population,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1B,66108.0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Fast Food Restaurant,Accessories Store,Museum
1,M1C,35626.0,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Bar,Accessories Store,Modern European Restaurant
2,M1E,46943.0,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Medical Center,Restaurant,Breakfast Spot
3,M1H,24383.0,Scarborough,Cedarbrae,43.773136,-79.239476,Bakery,Fried Chicken Joint,Gas Station
4,M1J,36699.0,Scarborough,Scarborough Village,43.744734,-79.239476,Playground,Accessories Store,Miscellaneous Shop


In [29]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)
map_frame(df)
map

More neighborhoods cut out again, lowering our options from 81 potential neighborhoods down to 73.

## Analysis C - Drop neighborhoods with coffee shops as their 3rd most common venue type
The first obvious step is to drop any neighborhood already dominated by coffee shops from our options.

Check shapes

First, our data after Analysis B

In [30]:
df.shape

(73, 9)

Next, after dropping more bad locations

In [31]:
df[~df['3rd Most Common Venue'].isin(['Coffee Shop'])].shape

(70, 9)

Update the dataframe again.

In [32]:
df = df[~df['3rd Most Common Venue'].isin(['Coffee Shop'])]
df = df.reset_index()
df = df.drop(columns='index')
df.head()

,Postal Code,Population,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M1B,66108.0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Fast Food Restaurant,Accessories Store,Museum
1,M1C,35626.0,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Bar,Accessories Store,Modern European Restaurant
2,M1E,46943.0,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Medical Center,Restaurant,Breakfast Spot
3,M1H,24383.0,Scarborough,Cedarbrae,43.773136,-79.239476,Bakery,Fried Chicken Joint,Gas Station
4,M1J,36699.0,Scarborough,Scarborough Village,43.744734,-79.239476,Playground,Accessories Store,Miscellaneous Shop


In [33]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)
map_frame(df)
map

More neighborhoods cut out again, lowering our options from 73 potential neighborhoods down to 70.

## Decision Basis
Lets reorder our dataframe by population to find our best neighbohood candidates

In [39]:
df = df.sort_values('Population', ascending=False)
df = df.reset_index()
df = df.drop(columns='level_0')
df.head()

,Postal Code,Population,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M2N,75897.0,North York,Willowdale South,43.770120,-79.408493,Ramen Restaurant,Pizza Place,Café
1,M1B,66108.0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,Fast Food Restaurant,Accessories Store,Museum
2,M9V,55959.0,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,Grocery Store,Pharmacy,Video Store
3,M1V,54680.0,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577,Intersection,Playground,Park
4,M5V,49195.0,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Airport Service,Airport Lounge,Airport Terminal


So above is the best choices available to us. The top 5 ideal neighborhoods in toronto to open a new coffee shop are:

    South Willowdale
    Malvern, Rouge
    South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens
    Milliken / Agincourt North / Steeles East / L'Amoreaux East
    CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / YTZ
    
All featuring the highest potential customer base (population) and lowest competition (coffee shops do not occupy and of the top 3 venue categories).

# Results & Discussion - The best choice <a name="RD"></a>

Thanks to our analysis, it is clear now the absolute best location to open a new coffee shop based on population and competition is Willowdale South. With a considerable population and very little competition, this neighborhood offers a great development opportunity for a new coffee shop.

It's worth noting, however, that cafes do however occupy the space as the 3rd most common venue type in **South Willodale**. Although a different type of business, cafes are dangerously similar, so other strong alternative locations include Malvern Rouge, and the areas M9V, M1V, and M5V.

Although a good start we could grealty improve this analysis in a number of ways. First, as you noticed throughout the methodology section, we had to drop a number of neighborhoods due to uneven data. Ideally we would avoid this either by finding stronger data to start with, or manually injecting the information we lost over the data treatment process. Furthermore, the accuracy of our decision could also have been improved greatly by considering more factors in our analysis. For example, rather than only concerning ourselves with population and competition, we could also factor in cost of operation, traffic areas, etc. 

# Conclusion - What's next? <a name="C"></a>

The purpose of this analysis was to identify neighborhoods in Toronto that are prime locations to open a new coffee shop considering two factors; potential customer base, and local competition levels. Using census data, wikipedia information, and the foursquare API, we came to the conculsion that **the best choice is Willowdale South**.

Next steps include improving our data collection and handling to avoid cutting any neghborhoods out of our analysis, and increasing the number of factors we consider for our conclusion.